In [ ]:
import random
import numpy
import json
import copy

In [ ]:
def findKey(dictionary, value):
    keys = []
    for key, val in dictionary.items():
        if val == value:
            keys.append(key)
    return keys


In [ ]:
# Hardcoded binary encoding for days
encodedDays = {
    'Monday': '000',
    'Tuesday': '001',
    'Wednesday': '010',
    'Thursday': '011',
    'Friday': '100'
}

# Time slots
timeSlots = {
    'Monday' : ['8:30-9:50','10:00-11:20','11:30-12:50','1:00-2:20','2:30-3:50','4:00-5:20'],
    'Tuesday' : ['8:30-9:50','10:00-11:20','11:30-12:50','1:00-2:20','2:30-3:50','4:00-5:20'],
    'Wednesday' : ['8:30-9:50','10:00-11:20','11:30-12:50','1:00-2:20','2:30-3:50','4:00-5:20'],
    'Thursday' : ['8:30-9:50','10:00-11:20','11:30-12:50','1:00-2:20','2:30-3:50','4:00-5:20'],
    'Friday' : ['8:30-9:50','10:00-11:20','11:30-12:50','1:00-2:20','2:30-3:50','4:00-5:20']
}

# Binary encoding for each time slot
encodedSlots = {}
count = 0
for day, slots in timeSlots.items():
    count = 0
    for slot in slots:
        encodedSlots[day + '-' + slot] = encodedDays[day] + format(count, 'b').zfill(3)
        count += 1

In [ ]:
# List of rooms
rooms = [
    'C-301', 'C-302', 'C-303', 'C-304', 'C-305', 'C-307', 'C-308', 'C-309', 'C-310', 'C-311',
    'C-401', 'C-402', 'C-403', 'C-404', 'C-405', 'C-406', 'C-407', 'C-408', 'C-409', 'C-410',
    'B-130', 'B-227', 'B-229', 'B-230',
    'A-108', 'A-211', 'A-301', 'A-302', 'A-303', 'A-305', 'A-310', 'A-311', 'A-314', 'A-315', 'A-316',
    'C-110', 'A-118 (MEDC)'
]

rooms = [room + '  60' for room in rooms]


# List of labs
labs = [
    'C-Margala 1', 'C-Margala 3', 'C-Margala 4', 'C-Rawal 1', 'Rawal 3 (B-232)', 'C-Rawal 4', 'C-GPU Lab',
    'A-Karakoram 1', 'A-Karakoram 2', 'A-Karakoram 3', 'A-Mehran 1', 'A-Mehran 2', 'A-CALL-1', 'A-CALL-2', 'A-CALL-3', 'B-Digital'
]
labs= [lab + '  80' for lab in labs]
# Binary encoding for rooms
room_binary_encoding = {}
for i, room in enumerate(rooms):
    room_binary_encoding[room] = '0' + format(i, 'b').zfill(6)

# Binary encoding for labs
lab_binary_encoding = {}
for i, lab in enumerate(labs):
    lab_binary_encoding[lab] = '1' + format(i, 'b').zfill(6)

# Combine room and lab encodings
encodedRooms = {**room_binary_encoding, **lab_binary_encoding}
print(lab_binary_encoding)

{'C-Margala 1  80': '1000000', 'C-Margala 3  80': '1000001', 'C-Margala 4  80': '1000010', 'C-Rawal 1  80': '1000011', 'Rawal 3 (B-232)  80': '1000100', 'C-Rawal 4  80': '1000101', 'C-GPU Lab  80': '1000110', 'A-Karakoram 1  80': '1000111', 'A-Karakoram 2  80': '1001000', 'A-Karakoram 3  80': '1001001', 'A-Mehran 1  80': '1001010', 'A-Mehran 2  80': '1001011', 'A-CALL-1  80': '1001100', 'A-CALL-2  80': '1001101', 'A-CALL-3  80': '1001110', 'B-Digital  80': '1001111'}


In [ ]:
def process_data(file_path):
    # Read the data from the file
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Initialize an empty list to store the data
    data = []
    current_course = None

    # Process each line and split it into course name, section, and instructor
    for line in lines:
        if '(' in line and ')' in line:
            # New course starts
            current_course = line.strip()
        else:
            parts = line.strip().split('\t')
            if len(parts) >= 2:  # Check if there are enough elements
                section = parts[0]
                instructor = parts[1]
                if current_course:
                    course_name = current_course
                    type = "Theory"
                    if "Lab" in course_name:
                        type = "Lab"
                    data.append([course_name,type, section, instructor])
            else:
                print("Invalid line format:", line)

    return data


In [ ]:
result = process_data("data.txt")
data = [' '.join(entry) for entry in result]

encodedCourses = {}
for i, room in enumerate(data):
    if room.find('lab')!=-1 or room.find('Lab')!=-1:
        encodedCourses[room] =format(i, 'b').zfill(10) + '0'
    else:
        encodedCourses[room] =format(i, 'b').zfill(10) + '1'

Invalid line format: 



In [ ]:
def createChromosomes(courses,slots,rooms,labs):
    chromosome = []
    for key3, value3 in courses.items():
        lab = False
        if value3[-1]=='0':
            lab = True
        # Randomly select a value from dict1 and dict2
        if lab==False:
            random_value1 = random.choice(list(rooms.values()))
            random_value2 = random.choice(list(slots.values()))
            random_value3 = random.choice(list(rooms.values()))
            random_value4 = random.choice(list(slots.values()))
            combined_string = f"{random_value1} {random_value2} {value3} {random_value3} {random_value4}"
            chromosome.append(combined_string)
        else:
            random_value1 = random.choice(list(labs.values()))
            random_value2 = random.choice(list(slots.values()))
            random_value3 = random.choice(list(labs.values()))
            random_value4 = random.choice(list(slots.values()))
            combined_string = f"{random_value1} {random_value2} {value3} {random_value3} {random_value4}"
            chromosome.append(combined_string)
    return chromosome


In [ ]:
def decode_chromosomes(chromosomes, courses, slots, rooms):
    decoded_values = []
    for chromosome in chromosomes:
        room, slot, course, room2, slot2 = chromosome.split()
        course_key = None
        slot_key = None
        room_key = None
        slot_key2 = None
        room_key2 = None

        # Find the key for course
        for key, value in courses.items():
            if value == course:
                course_key = key
                break

        # Find the key for slot
        for key, value in slots.items():
            if value == slot:
                slot_key = key
                break

        # Find the key for room
        for key, value in rooms.items():
            if value=='0':
                break
            if value == room:
                room_key = key
                break

                # Find the key for slot2
        for key, value in slots.items():
            if value == slot2:
                slot_key2 = key
                break

        # Find the key for room2
        for key, value in rooms.items():
            if value=='0':
                break
            if value == room2:
                room_key2 = key
                break



        decoded_values.append(f"{course_key}  {room_key}  {slot_key}  {slot_key2}  {room_key2}")

    return decoded_values


In [ ]:
chromosomes = createChromosomes(encodedCourses,encodedSlots,room_binary_encoding,lab_binary_encoding)
decoded= decode_chromosomes(chromosomes,encodedCourses,encodedSlots,encodedRooms)

In [ ]:
population=[]
for i in range(20):
    chromosome=[]
    chromosome = createChromosomes(encodedCourses,encodedSlots,room_binary_encoding,lab_binary_encoding)
    population.append((chromosome,0))

**Now that the chromosomes are creates, we will start the genetic algorithm code**

In [ ]:
def fitness(pop):
    fit =[]
    for i, (chromosomes, _) in enumerate(pop):
        total_fitness = 0  # Initialize total fitness for the current chromosome
        for classes in chromosomes:
            total_fitness += calcFitness(classes, chromosomes)
        # Update fitness for the current chromosome in the population
        pop[i] = (chromosomes, total_fitness)
        fit.append(total_fitness)

    return fit

def calcFitness(chromo,pop):
    room1, slot1, course, room2, slot2 = chromo.split()
    fitness = 0
    day1 = slot1[:3]
    day2 = slot2[:3]


    if(room1 !=room2):
        fitness!=1

    if(course[-1]=='0'):
        if (room1!=room2):
            fitness+=10 #add 10 if lab not in consecutive slot
        if (abs(int(slot1,2)-int(slot2,2))!=1):
            fitness+=10
        if int(slot1[3:],2)<=3: #soft constraint minus 1 if lab class in afternoon
            fitness+=1
        if int(slot2[3:],2)<=3:
            fitness+=1
    else:
        if (day1=='000' and day2 == '000') or (day1=='001' and day2 == '001') or (day1=='010' and day2 == '010') or (day1=='011' and day2 == '011') or (day1=='100' and day2 == '100'):
            fitness+=10 #add 10 if both slots are on same days and its not a lab
        if (day1=='000' and day2 == '001') or (day1=='001' and day2 == '010') or (day1=='010' and day2 == '011') or (day1=='011' and day2 == '100'):
            fitness+=10 #add 10 if both slots are on adjascent days
        if int(slot1[3:],2)>3: #soft constraint minus 1 if theory class in morning
            fitness+=1
        if int(slot2[3:],2)>3:
            fitness+=1
    for chromos in pop:
        r1, s1, c, r2, s2 = chromos.split()
        if chromos==chromo:
            continue
        if r1==room1 and s1 ==slot1:
            fitness+=10 #add 10 if clash
        if r2==room2 and s2 ==slot2:
            fitness+=10 #add 10 if clash
    return fitness



def selection(pop):
    sorted_list = sorted(population, key=lambda x: x[1])
    x1=[]
    x2=[]
    x3=[]
    x4=[]
    x1 = sorted_list[0]
    x2 = sorted_list[1]
    x3 = sorted_list[-2]
    x4 = sorted_list[-1]
    return x1,x2,x3,x4


def crossover(x1, x2):
    # Extract the lists from the tuples and make copies
    list1 = copy.deepcopy(x1[0])
    list2 = copy.deepcopy(x2[0])

    # Select 10 random indices from each list
    indices1 = random.sample(range(len(list1)), 150)
    indices2 = random.sample(range(len(list2)), 150)

    # Exchange the elements at the selected indices
    for i in range(150):
        index1 = indices1[i]
        index2 = indices2[i]
        list1[index1], list2[index2] = list2[index2], list1[index1]

    # Return the modified tuples
    return (list1, x1[1]), (list2, x2[1])

def mutate(x1, lab,room):
    # Make a copy of the chromosome list
    chromo = copy.deepcopy(x1[0])

    # Perform mutation on the copied chromosome
    indices1 = random.sample(range(len(chromo)), 1)
    for i in indices1:

        r1, s1, c, _, s2 = chromo.pop(random.randint(0, 395)).split()
        labs = False
        if c[-1]=='0':
            labs = True
        if labs == True:
            random_value1 = random.choice(list(lab.values()))
        else:
            random_value1 = random.choice(list(room.values()))

        combined_string = f"{r1} {s1} {c} {random_value1} {s2}"
        chromo.insert(i, combined_string)

    return (chromo, x1[1])


In [ ]:

def replace_population(old_population, new_population):
    # Replace some individuals in the old population with new offspring
    pass

# Main genetic algorithm function
def genetic_algorithm(max_generations):


    for i in range(max_generations):
        # Evaluate fitness
        fitness_scores=[]
        fitness_scores = sorted(fitness(population))
        if fitness_scores[0]==0:
            break
        print("Generation: ",i)
        print("Fitness Scores: ",fitness_scores)
        parent1,parent2,x1,x2 = selection(population)


        child1,child2 = crossover(parent1,parent2)
        mchild1 = mutate(child1,lab_binary_encoding,room_binary_encoding)
        mchild2 = mutate(child2,lab_binary_encoding,room_binary_encoding)

        population.remove(x1)
        population.remove(x2)
        population.append(mchild1)
        population.append(mchild2)

genetic_algorithm(max_generations=1500)


Generation:  0
Fitness Scores:  [5198, 5408, 5452, 5520, 5600, 5606, 5657, 5665, 5718, 5721, 5756, 5765, 5770, 5856, 5889, 5929, 5952, 5955, 5960, 6113]
Generation:  1
Fitness Scores:  [5198, 5408, 5452, 5520, 5537, 5600, 5606, 5657, 5665, 5669, 5718, 5721, 5756, 5765, 5770, 5856, 5889, 5929, 5952, 5955]
Generation:  2
Fitness Scores:  [5198, 5408, 5452, 5520, 5537, 5563, 5600, 5606, 5657, 5665, 5669, 5693, 5718, 5721, 5756, 5765, 5770, 5856, 5889, 5929]
Generation:  3
Fitness Scores:  [5198, 5350, 5408, 5452, 5520, 5537, 5563, 5600, 5606, 5656, 5657, 5665, 5669, 5693, 5718, 5721, 5756, 5765, 5770, 5856]
Generation:  4
Fitness Scores:  [5198, 5276, 5350, 5408, 5452, 5520, 5532, 5537, 5563, 5600, 5606, 5656, 5657, 5665, 5669, 5693, 5718, 5721, 5756, 5765]
Generation:  5
Fitness Scores:  [4967, 5198, 5276, 5350, 5408, 5452, 5520, 5532, 5537, 5537, 5563, 5600, 5606, 5656, 5657, 5665, 5669, 5693, 5718, 5721]
Generation:  6
Fitness Scores:  [4967, 5107, 5118, 5198, 5276, 5350, 5408, 5452, 5

KeyboardInterrupt: 

In [ ]:

def replace_population(old_population, new_population):
    # Replace some individuals in the old population with new offspring
    pass

# Main genetic algorithm function
def genetic_algorithm(max_generations):


    for i in range(max_generations):
        # Evaluate fitness
        fitness_scores=[]
        fitness_scores = sorted(fitness(population))
        if fitness_scores[0]==0:
            break
        print("Generation: ",i)
        print("Fitness Scores: ",fitness_scores)
        # Selection
        #print("Selection")
        parent1,parent2,x1,x2 = selection(population)

        # Crossover
       # print("Crossover")
        child1,child2 = crossover(parent1,parent2)
        mchild1 = mutate(child1,lab_binary_encoding,room_binary_encoding)
        mchild2 = mutate(child2,lab_binary_encoding,room_binary_encoding)

        population.remove(x1)
        population.remove(x2)
        population.append(mchild1)
        population.append(mchild2)



# Example usage
genetic_algorithm(max_generations=1500)


Generation:  0
Fitness Scores:  [436, 445, 446, 448, 467, 467, 469, 475, 482, 483, 490, 495, 497, 498, 500, 508, 509, 518, 1975, 1986]
Generation:  1
Fitness Scores:  [436, 445, 446, 448, 467, 467, 469, 475, 482, 483, 490, 495, 497, 498, 500, 508, 509, 518, 906, 915]
Generation:  2
Fitness Scores:  [436, 445, 446, 448, 467, 467, 467, 469, 475, 482, 483, 490, 495, 497, 498, 500, 508, 509, 518, 2034]
Generation:  3
Fitness Scores:  [436, 445, 446, 448, 467, 467, 467, 469, 475, 482, 483, 490, 495, 497, 498, 500, 508, 509, 677, 1004]
Generation:  4
Fitness Scores:  [436, 445, 446, 448, 467, 467, 467, 469, 475, 482, 483, 490, 495, 497, 498, 500, 508, 509, 692, 1029]
Generation:  5
Fitness Scores:  [436, 445, 446, 448, 467, 467, 467, 469, 475, 482, 483, 490, 495, 497, 498, 500, 508, 509, 694, 847]
Generation:  6
Fitness Scores:  [436, 445, 446, 448, 467, 467, 467, 469, 475, 482, 483, 490, 495, 497, 498, 500, 508, 509, 579, 1082]
Generation:  7
Fitness Scores:  [436, 445, 446, 448, 467, 467, 

KeyboardInterrupt: 

In [ ]:
d= []
sorted_list = sorted(population, key=lambda x: x[1])
print(sorted_list[1])
for i in sorted_list[0]:

    d.append(decode_chromosomes(i,encodedCourses,encodedSlots,encodedRooms))


(['0011011 011000 00000001111 0100000 010001', '0010001 010010 00110001111 0001101 000001', '0000000 011101 00100011101 0100001 011010', '0001111 100010 00010001111 0011011 001101', '0001001 100101 00111011001 0010101 001001', '0001001 100101 00111011001 0010101 001001', '0010001 010010 00110001111 0001101 000001', '0001100 100010 00110101011 0010000 011010', '0001110 100011 00110000101 0100010 010011', '0001111 100010 00010001111 0011011 001101', '0010001 010010 00110001111 0001101 000001', '0000100 010101 00011111011 0001110 100010', '0010001 010010 00110001111 0001101 000001', '0011011 011011 00101001011 0011001 000001', '0011011 011011 00101001011 0011001 000001', '0000100 010101 00011111011 0001110 100010', '0100011 001000 00100010111 0010110 011011', '0000100 010101 00011111011 0001110 100010', '0000100 010101 00011111011 0001110 100010', '1001101 000101 01001100010 1000110 000100', '0011010 011010 01000100011 0000110 010010', '0011011 011011 00101001011 0011001 000001', '0010001

TypeError: 'int' object is not iterable

In [ ]:
with open('output.txt', 'a+') as file:
    # Read existing content to check for duplicates
    file.seek(0)  # Move the cursor to the beginning of the file
    existing_content = file.readlines()
    #existing_content = [line.strip() for line in existing_content]  # Remove newline characters

    # Filter out duplicates from the data
    unique_data = [line for line in d[0] if line not in existing_content]

    # Write only unique data to the file
    for line in unique_data:
        file.write(line + '\n')
